In [12]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from helpers import dataframe
from tensorflow import keras
import numpy as np
from keras import layers, models

In [3]:
base_url = "https://raw.githubusercontent.com/italha-d/Stress-Predict-Dataset/main/Raw_data"

# subjects = ["S01", "S02", "S03"]
subjects = [
    "S01", "S02", "S03", "S04", "S05", "S06", "S07", "S08", "S09", "S10",
    # "S11", "S12", "S13", "S14", "S15", "S16", "S17", "S18", "S19", "S20",
    # "S21", "S22", "S23", "S24", "S25", "S26", "S27", "S28", "S29", "S30",
    # "S31", "S32", "S33", "S34", "S35"
]

fetch_objects = dataframe.generate_fetch_objects(base_url, subjects)

In [4]:
acc_df, hr_df, eda_df, temp_df, bvp_df = dataframe.concatenate_dataframe_from_fetch_objects(fetch_objects)

df = dataframe.combine_dataframe(acc_df, hr_df, eda_df, temp_df, bvp_df)

df = dataframe.fill_missing_values(df)

Fetching HR.csv for subject S01
Fetching ACC.csv for subject S01
Fetching BVP.csv for subject S01
Fetching EDA.csv for subject S01
Fetching TEMP.csv for subject S01
Fetching HR.csv for subject S02
Fetching ACC.csv for subject S02
Fetching BVP.csv for subject S02
Fetching EDA.csv for subject S02
Fetching TEMP.csv for subject S02
Fetching HR.csv for subject S03
Fetching ACC.csv for subject S03
Fetching BVP.csv for subject S03
Fetching EDA.csv for subject S03
Fetching TEMP.csv for subject S03
Fetching HR.csv for subject S04
Fetching ACC.csv for subject S04
Fetching BVP.csv for subject S04
Fetching EDA.csv for subject S04
Fetching TEMP.csv for subject S04
Fetching HR.csv for subject S05
Fetching ACC.csv for subject S05
Fetching BVP.csv for subject S05
Fetching EDA.csv for subject S05
Fetching TEMP.csv for subject S05
Fetching HR.csv for subject S06
Fetching ACC.csv for subject S06
Fetching BVP.csv for subject S06
Fetching EDA.csv for subject S06
Fetching TEMP.csv for subject S06
Fetching H

In [5]:
tags_object = dataframe.generate_tag_objects(base_url, subjects)

timestamps = dataframe.fetch_timestamps_from_tag_objects(tags_object)

df = dataframe.label_dataframe_from_timestamps(df, timestamps)

In [6]:
df.to_csv("data.csv", index=False)

KeyboardInterrupt: ignored

### Subject division

In [7]:
new_df = df.set_index(['Subject', 'Timestamp'])

new_df

X     Y     Z     HR       EDA   TEMP     BVP  Label
Subject Timestamp                                                           
1.0     1.644226e+12 -2.0  43.0  43.0  83.00  0.000000  28.39    0.00    0.0
        1.644226e+12 -2.0  44.0  46.0  83.00  0.000000  28.39    0.00    0.0
        1.644226e+12  2.0  44.0  47.0  83.00  0.000000  28.39    0.00    0.0
        1.644226e+12  3.0  43.0  44.0  83.00  0.000000  28.39    0.00    0.0
        1.644226e+12  1.0  43.0  45.0  83.00  0.000000  28.39    0.00    0.0
...                   ...   ...   ...    ...       ...    ...     ...    ...
10.0    1.644847e+12  3.0  -3.0  63.0  76.53  0.306487  26.89 -115.03    0.0
        1.644847e+12  3.0  -3.0  63.0  76.53  0.306487  26.89 -129.06    0.0
        1.644847e+12  3.0  -3.0  63.0  76.53  0.306487  26.89 -144.23    0.0
        1.644847e+12  3.0  -3.0  63.0  76.53  0.306487  26.89 -155.62    0.0
        1.644847e+12  3.0  -3.0  63.0  76.53  0.306487  26.89 -158.14    0.0

[2113911 rows x 8 columns]

In [6]:
from sklearn.model_selection import train_test_split

train_subjects, test_subjects = train_test_split(new_df.index.get_level_values('Subject').unique(), test_size=0.2)

In [ ]:
from sklearn.linear_model import LogisticRegression

logistic_regression_model = LogisticRegression()
logistic_regression_model.fit(new_df.loc[train_subjects].drop('Label', axis=1), new_df.loc[train_subjects]['Label'])

logistic_regression_model.score(new_df.loc[test_subjects].drop('Label', axis=1), new_df.loc[test_subjects]['Label'])

0.7083988864575852

In [8]:
from keras.preprocessing.sequence import TimeseriesGenerator

X_train = new_df.loc[train_subjects].drop('Label', axis=1).values
y_train = new_df.loc[train_subjects]['Label'].values
X_test = new_df.loc[test_subjects].drop('Label', axis=1).values
y_test = new_df.loc[test_subjects]['Label'].values

seq_length = 5
batch_size = 32

train_seq = TimeseriesGenerator(X_train, y_train, length=seq_length, batch_size=batch_size)

In [9]:
from keras import layers, models

model = models.Sequential()
model.add(layers.LSTM(64, input_shape=(seq_length, X_train.shape[1])))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(train_seq, epochs=2)

Epoch 1/2
53399/53399 [==============================] - 366s 7ms/step - loss: 0.3228 - accuracy: 0.8215
Epoch 2/2
53399/53399 [==============================] - 333s 6ms/step - loss: 0.2848 - accuracy: 0.8526


In [10]:
test_seq = TimeseriesGenerator(X_test, y_test, length=seq_length, batch_size=batch_size)
score = model.evaluate(test_seq)
print('Model accuracy:', score[1])

12662/12662 [==============================] - 48s 4ms/step - loss: 0.4273 - accuracy: 0.7840
Model accuracy: 0.7840373516082764


### Cross Validation

In [20]:
from sklearn.model_selection import KFold

# Define number of folds for cross-validation
n_folds = 5

# Initialize KFold object for splitting data by subjects
subject_kfold = KFold(n_splits=n_folds, shuffle=True)

# Iterate over folds and train model on each fold
for fold, (train_subjects, test_subjects) in enumerate(subject_kfold.split(df['Subject'].unique())):
    
    # Split data by subjects
    train_data = df[df['Subject'].isin(train_subjects)]
    test_data = df[df['Subject'].isin(test_subjects)]
    
    # Extract X and y for training and testing
    X_train = train_data.drop(['Subject', 'Label'], axis=1)
    y_train = train_data['Label']
    X_test = test_data.drop(['Subject', 'Label'], axis=1)
    y_test = test_data['Label']
    
    # Define your model architecture
    model = models.Sequential()
    model.add(layers.Dense(10, activation='relu', input_shape=(X_train.shape[1],)))
    model.add(layers.Dense(1, activation='sigmoid'))

    # Compile model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Train model on training data
    model.fit(X_train, y_train, epochs=1, batch_size=32)
    
    # Evaluate model on testing data
    score = model.evaluate(X_test, y_test)
    print('Fold {}: {}'.format(fold, score))

13160/13160 [==============================] - 26s 2ms/step - loss: 41507312.0000 - accuracy: 0.7503
Fold 0: [41507312.0, 0.750315248966217]
13194/13194 [==============================] - 23s 2ms/step - loss: 0.5805 - accuracy: 0.7328
Fold 1: [0.5805368423461914, 0.7328226566314697]
 6454/45775 [===>..........................] - ETA: 1:29 - loss: 2281726464.0000 - accuracy: 0.6225

KeyboardInterrupt: ignored

In [28]:
from sklearn.model_selection import KFold

n_folds = 5
seq_length = 5

subject_kfold = KFold(n_splits=n_folds, shuffle=True)

for fold, (train_subjects, test_subjects) in enumerate(subject_kfold.split(df['Subject'].unique())):
    train_data = df[df['Subject'].isin(train_subjects)]
    test_data = df[df['Subject'].isin(test_subjects)]

    train_dataset = keras.utils.timeseries_dataset_from_array(
      train_data,
      targets=train_data['Label'],
      sampling_rate=1,
      sequence_length=seq_length,
      batch_size=batch_size,
    )

    test_dataset = keras.utils.timeseries_dataset_from_array(
      test_data,
      targets=test_data['Label'],
      sampling_rate=1,
      sequence_length=seq_length,
      batch_size=batch_size,
    )
    
    model = models.Sequential()
    model.add(layers.LSTM(10, input_shape=(seq_length, len(df.columns))))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_dataset, epochs=1, batch_size=32)
    
    score = model.evaluate(test_dataset)
    print('Fold {}: {}'.format(fold, score))

7080/7080 [==============================] - 48s 7ms/step - loss: 4.2227 - accuracy: 0.7284
Fold 0: [4.222671985626221, 0.7283550500869751]
13399/13399 [==============================] - 92s 7ms/step - loss: 3.2831 - accuracy: 0.7640
Fold 1: [3.2831389904022217, 0.7640001177787781]
27005/46298 [================>.............] - ETA: 3:13 - loss: 0.5056 - accuracy: 0.7327

KeyboardInterrupt: ignored